In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import ExploratoryAnalysis.data_processing_modules as dpm

/Users/nick/miniforge3/envs/recSysEnv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Need to do some preprocessing on embeddings so that they get taken out of a column, expanded, and then applied to the list
news = pd.read_csv('MIND_small/csv/news.csv')
news.drop(columns=['Unnamed: 0'], inplace=True)
news = dpm.create_text_embeddings(news)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

The Battles began on The Voice Monday, with a new set of mentors (Usher, Darius Rucker, Normani, and wil.i.am) and a new rule allowing coaches to save one contestant each from elimination. But before all that, there were a few more Blind Auditions, including one form Season 17's new heartthrob, Matthew McQueen. He may not be related to Steve McQueen, but he had undeniable star power -- with a pretty face and a pretty voice to match plus just the right about of darkness and edge. "He kind of looks like a combination of Tom Cruise and Christian Bale. You're like a very handsome movie villain," John Legend told Matthew. "Like a vampire. You could be in a vampire movie," said Kelly Clarkson, who seemed a little flustered as she called the 21-year-old indie-pop crooner "so cool and sexy -- I mean, not in a weird way!" Blake Shelton compared Matthew's modern-sounding vocals to that of another heartthrob, Shawn Mendes. So I guess music really was in Matthew's, um, blood. [insert/quote some tw

In [ ]:
news.to_csv('MIND_small/csv/news_with_embedding_cols.csv')

In [ ]:
### Now we need to extract the embeddings into separate columns
abstract_embeddings = pd.DataFrame(news['abstract_embeddings'].to_list(), index=news.index)
title_embeddings = pd.DataFrame(news['title_embeddings'].to_list(), index=news.index)
abstract_embeddings.to_csv('MIND_small/csv/abstract_embeddings.csv')
title_embeddings.to_csv('MIND_small/csv/title_embeddings.csv')

In [ ]:
news = pd.concat([news, abstract_embeddings, title_embeddings], axis=1)
news

In [ ]:
del abstract_embeddings, title_embeddings